In [1]:
ticker = "QQQ"
TESTING = True

In [2]:
API_KEY =  '0MXZO7Y5VXKK2R3PXN5XH8EZEWJTG6BJ' + '@AMER.OAUTHAP'
ACCT_NUMBER = '255612401'
CALLBACK_URL = 'https://localhost:8888'


In [3]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random


from tda import auth, client
from tda.orders.equities import equity_buy_market, equity_sell_market
from tda.orders.common import Duration, Session
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
from datetime import datetime
import json
import schedule
import time
from dateutil.relativedelta import relativedelta

sns.set()


In [4]:
import pkg_resources
import types

def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split('.')[0]
        elif isinstance(val, type):
            name = val.__module__.split('.')[0]
        poorly_named_packages = {'PIL': 'Pillow', 'sklearn': 'scikit-learn'}
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
        yield name


imports = list(set(get_imports()))
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name != 'pip':
        requirements.append((m.project_name, m.version))

#for r in requirements:
#    print('{}=={}'.format(*r))

matplotlib==3.6.2
numpy==1.23.4
pandas==1.5.1
schedule==1.1.0
seaborn==0.12.1


In [ ]:
#df = pd.read_csv('.\\Data\\' + ticker + '.csv')
#testing = df.copy()

#if TESTING == True:
    #tot = len(df)
    #test_count = int(tot * .1)
    #train_count = int(tot * .9)
    #df = df.head(train_count)
    #testing = testing.tail(test_count)

#print(df.head())
#print(testing.head())

In [5]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function, population_size, sigma, learning_rate
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population)
            rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-7)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (
                    w
                    + self.learning_rate
                    / (self.population_size * self.sigma)
                    * np.dot(A.T, rewards).T
                )
            if (i + 1) % print_every == 0:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, self.reward_function(self.weights))
                )
        print('time taken to train:', time.time() - lasttime, 'seconds')


class Model:
    def __init__(self, input_size, layer_size, output_size):
        self.weights = [
            np.random.randn(input_size, layer_size),
            np.random.randn(layer_size, output_size),
            np.random.randn(1, layer_size),
        ]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        return decision

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [18]:
class Agent:

    POPULATION_SIZE = 90 #15
    SIGMA = 0.1
    LEARNING_RATE = 0.03

    def __init__(self, model, window_size, trend, skip, initial_money):
        self.model = model
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        self.initial_money = initial_money
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
        )

    
    def act(self, sequence):
        decision = self.model.predict(np.array(sequence))
        return np.argmax(decision[0])
    
    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array([res])

    def get_reward(self, weights):
        initial_money = self.initial_money
        starting_money = initial_money
        self.model.weights = weights
        state = self.get_state(0)
        inventory = []
        quantity = 0
        for t in range(0, len(self.trend) - 1, self.skip):
            action = self.act(state)
            next_state = self.get_state(t + 1)
            
            if action == 1 and starting_money >= self.trend[t]:
                inventory.append(self.trend[t])
                starting_money -= close[t]
                
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                starting_money += self.trend[t]

            state = next_state
        return ((starting_money - initial_money) / initial_money) * 100

    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

    def buy(self,  current_inventory, current_price, days_since_training):
        initial_money = self.initial_money
        state = self.get_state(days_since_training)
        starting_money = initial_money
        sell = False
        buy = False
    
        inventory = current_inventory
        action = self.act(state)
        
        if action == 1 and initial_money >= current_price: #self.trend[t]:
            inventory.append(current_price)
            initial_money -= current_price
            print('buy 1 unit at price %f, total balance %f'% (current_price, initial_money))
            buy = True
        elif action == 2 and len(inventory):
            bought_price = inventory.pop(0)
            initial_money += current_price#self.trend[t]
            try:
                invest = ((current_price - bought_price) / bought_price) * 100
            except:
                invest = 0
            print('sell 1 unit at price %f, investment %f %%, total balance %f,' % (current_price, invest, initial_money) )
            sell = True

        invest = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        return inventory, buy, sell, total_gains, invest

In [19]:
class TDAmeritrade:
    
    def __init__(self, account_number, api_key, callback_url, symbol):
        token_path = 'token.pickle'
        self.SYMBOL = symbol
        self.API_KEY = api_key
        self.CALLBACK_URL = callback_url
        self.ACCOUNT_NUMBER = account_number
        try:
            self.c = auth.client_from_token_file(token_path, self.API_KEY)
        except FileNotFoundError:
            from selenium import webdriver
            with webdriver.Chrome(ChromeDriverManager().install()) as driver:
                self.c = auth.client_from_login_flow(
                    driver, self.API_KEY, self.CALLBACK_URL, token_path)

        return
    
    def get_prices(self, end):

        # fetch price history using tda-api
        r = self.c.get_price_history(self.SYMBOL,
                                period_type=client.Client.PriceHistory.PeriodType.DAY,
                                period=client.Client.PriceHistory.Period.THREE_DAYS,
                                frequency_type=client.Client.PriceHistory.FrequencyType.MINUTE,
                                frequency=client.Client.PriceHistory.Frequency.EVERY_THIRTY_MINUTES,
                                end_datetime=end,
                                need_extended_hours_data=False
                                )

        assert r.status_code == 200, r.raise_for_status()

        # parse json and get candles data
        y = r.json()
        y = y["candles"]
        y = json.dumps(y)
        df = pd.read_json(y)
        # drop last row
        df = df[:-1]

        return df
    
    def get_daily_price_history(self, symbol, start_date, end_date):
        r = self.c.get_price_history_every_day(symbol=symbol, start_datetime=start_date, end_datetime=end_date)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        y = y["candles"]
        y = json.dumps(y)
       
        df = pd.read_json(y)        
        return df

    def get_cur_price(self):

        r = self.c.get_quote(self.SYMBOL)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        price = y[self.SYMBOL]["lastPrice"]

        return price

    # get useful account info


    def get_account(self):

        r = self.c.get_account(self.ACCOUNT_NUMBER)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()

        return y

    # see if we currently have a position
    def get_available_funds(self):
        y = self.get_account()
        return y['securitiesAccount']['currentBalances']['availableFunds']

    def get_has_position(self):
        y = self.get_account()
        
        if "positions" in y["securitiesAccount"]:
            return True
        else:
            return False

    def get_positions(self):
        y = self.get_account()

        if "positions" in y["securitiesAccount"]:
            return y["securitiesAccount"]["positions"]
        else:
            return []
        

    def place_order(self, order_type, shares):

        if order_type == 'buy':
            order_spec = equity_buy_market(self.SYMBOL, shares).set_session(
                Session.NORMAL).set_duration(Duration.DAY).build()
            self.c.place_order(self.ACCOUNT_NUMBER, order_spec)

        if order_type == 'sell':
            order_spec = equity_sell_market(self.SYMBOL, shares).set_session(
                Session.NORMAL).set_duration(Duration.DAY).build()
            self.c.place_order(self.ACCOUNT_NUMBER, order_spec)

    def get_transactions(self):
        r = self.c.get_transactions(self.ACCOUNT_NUMBER)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        
        return y

In [20]:
td_ameritrade = TDAmeritrade(ACCT_NUMBER, API_KEY, CALLBACK_URL, ticker)

current_date = datetime.today()
past_date = current_date - relativedelta(years=2)
#df = pd.read_csv('.\\Data\\' + ticker + '.csv')
df = td_ameritrade.get_daily_price_history(ticker, past_date, current_date)

df.rename(columns = {'datetime':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'}, inplace = True)
testing = df.copy()

if TESTING == True:
    tot = len(df)
    test_count = int(tot * .1)
    train_count = int(tot * .9)
    df = df.head(train_count)
    testing = testing.tail(test_count)
    #display(df)
    #display(testing)

In [21]:
close = df.Close.values.tolist()
window_size = 30
skip = 1

avail_funds = td_ameritrade.get_available_funds()
model = Model(input_size = window_size, layer_size = 500, output_size = 3)
agent = Agent(model = model, 
              window_size = window_size,
              trend = close,
              skip = skip,
              initial_money = avail_funds)
agent.fit(iterations = 200, checkpoint = 2)

iter 2. reward: -34.269048
iter 4. reward: -30.889474
iter 6. reward: -15.069063
iter 8. reward: -9.881004
iter 10. reward: -7.391548
iter 12. reward: -7.651248
iter 14. reward: -4.512046
iter 16. reward: -2.772946
iter 18. reward: 0.529313
iter 20. reward: 2.937489
iter 22. reward: 4.623558
iter 24. reward: 3.020256
iter 26. reward: 7.514955
iter 28. reward: 7.182400
iter 30. reward: 7.140273
iter 32. reward: 6.577754
iter 34. reward: 8.317845
iter 36. reward: 8.534923
iter 38. reward: 11.328684
iter 40. reward: 12.017584
iter 42. reward: 12.581094
iter 44. reward: 14.215125
iter 46. reward: 15.815454
iter 48. reward: 16.878045
iter 50. reward: 18.221647
iter 52. reward: 16.972211
iter 54. reward: 16.431003
iter 56. reward: 15.689073
iter 58. reward: 20.236307
iter 60. reward: 17.984745
iter 62. reward: 19.338755
iter 64. reward: 19.339251
iter 66. reward: 22.083947
iter 68. reward: 21.879259
iter 70. reward: 21.827716
iter 72. reward: 22.266332
iter 74. reward: 20.664516
iter 76. rew

In [22]:
def trade_if_ai_says_to():
    print('trade_if_ai_says_to')

    last_dt = df.iloc[-1]['Date']
    #last_dt = datetime.strptime(date_string, '%Y-%m-%d')
    now = datetime.now()
    countdown = now - last_dt
    days_since_training_the_model = countdown.days

    inventory = td_ameritrade.get_positions()
    cur_price = td_ameritrade.get_cur_price()
    
    print("Price: " + str(cur_price) + " Day: " + str(days_since_training_the_model))
    inventory, buy, sell, tg, invest = agent.buy(inventory, cur_price, days_since_training_the_model)

def test_trade_if_ai_says_to(index, inventory):
    last_dt = df.iloc[-1]['Date']
    now_dt = testing.iloc[index]['Date']
    countdown = now_dt - last_dt
    days_since_training_the_model = countdown.days

    cur_price = testing.iloc[index]['Open']
    
    print("Price: " + str(cur_price) + " Day: " + str(days_since_training_the_model))
    inventory, buy, sell, tg, invest = agent.buy(inventory, cur_price, days_since_training_the_model)

    return inventory

In [23]:

# TODO: Update Historical prices every-day?
# TODO: Retrain Every day?
# TODO: Figure out if the market is open

if TESTING == False:
    schedule.every().monday.at('07:30').do(trade_if_ai_says_to)
    schedule.every().tuesday.at('07:30').do(trade_if_ai_says_to)
    schedule.every().wednesday.at('07:30').do(trade_if_ai_says_to)
    schedule.every().thursday.at('07:30').do(trade_if_ai_says_to)
    schedule.every().friday.at('07:30').do(trade_if_ai_says_to)

    while True:
       schedule.run_pending()
       time.sleep(1)

elif TESTING == True:
    inventory = []
    rg = len(testing)
    for i in range(0, rg - 1):
        inventory = test_trade_if_ai_says_to(i, inventory)
        #print(inventory)



Price: 400.28 Day: 2
Price: 393.13 Day: 6
buy 1 unit at price 393.130000, total balance 1624.580000
Price: 390.43 Day: 7
buy 1 unit at price 390.430000, total balance 1627.280000
Price: 395.39 Day: 8
Price: 402.74 Day: 9
Price: 408.78 Day: 12
Price: 401.83 Day: 13
sell 1 unit at price 401.830000, investment 2.213008 %, total balance 2419.540000,
Price: 394.47 Day: 14
buy 1 unit at price 394.470000, total balance 1623.240000
Price: 392.96 Day: 15
sell 1 unit at price 392.960000, investment 0.648003 %, total balance 2410.670000,
Price: 384.14 Day: 16
sell 1 unit at price 384.140000, investment -2.618704 %, total balance 2401.850000,
Price: 382.26 Day: 19
Price: 385.06 Day: 20
buy 1 unit at price 385.060000, total balance 1632.650000
Price: 386.11 Day: 21
buy 1 unit at price 386.110000, total balance 1631.600000
Price: 376.58 Day: 22
sell 1 unit at price 376.580000, investment -2.202254 %, total balance 2394.290000,
Price: 370.58 Day: 23
sell 1 unit at price 370.580000, investment -4.0221

In [ ]:
#fig = plt.figure(figsize = (15,5))
#plt.plot(close, color='r', lw=2.)
#plt.plot(close, '^', markersize=10, color='m', label = 'buying signal', markevery = states_buy)
#plt.plot(close, 'v', markersize=10, color='k', label = 'selling signal', markevery = states_sell)
#plt.title('total gains %f, total investment %f%%'%(total_gains, invest))
#plt.legend()
#plt.show()